In [26]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [21]:
def dydx(x,y):
    #set the derivatives
    #our equation is d^2x/dx^2 = -y
    #so we can write
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0]
    #we will set z = y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #return arrays of dydx
    return y_derivs

In [22]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    
    #declare k? arrays (? any digit 0-9)
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    #define x step
    x_ipoh_k2 = xi + c2*h
    x_ipoh_k3 = xi + c3*h
    x_ipoh_k4 = xi + c4*h
    x_ipoh_k5 = xi + c5*h
    x_ipoh_k6 = xi + c6*h
    
    #define y at 1 step
    x_ipo_k2 = yi + a21k1
    x_ipo_k3 = yi + a31k1 + a32k2
    x_ipo_k4 = yi + a41k1 + a42k2 + a43k3
    x_ipo_k5 = yi + a51k1 + a42k2 + a53k3 + a54k4
    x_ipo_k6 = yi + a61k1 + a42k2 + a63k3 + a64k4 + a65k5
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get the k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k4[:] = h*y_derivs[:]
    
    #advance y by a step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    #this is an array (yipo = y i plus one)
    return yipo

In [23]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterarions
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust the step
    while(Delta.max()/tol > 1.0):
        
        #estimate our error by taking one step of size h vs. 2 steps of size h/2
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i + 0.5*h_step,y_1,nv,0.5*h_step)
        
        #compute an error
        Delta = np.fabs(y_2 - y_11)
        
        #if the error is too large, decrease the step
        if (Delta.max()/tol > 1.0):
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
           
        #check iterations
        if (i>=imax): 
            print("too many iterations in rk4_mv_ad()")
            raise StopIteration("ending after i =",i)
            
        #iterate
        i+= 1
        
    #next time try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, anew step, and the step we actually took
    return y_2, h_new, h_step

In [24]:
def rk4_mv(dydx,a,b,y_a,tol):
    
    #dydx is the derivative wrt x
    #a is lower bound and b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    
    #an intial step size == very small
    h = 1.0e-4 * (b-a)
    
    #set number 0f iterations
    imax = 10000
   
    #set  an iteration variable
    i = 0
    
    #set the number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set the numper of coupled odes to the size of y_a
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set initial conditions
    flag = 1
    
    #loop until we reach th right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if (xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            
            flag = 0
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        #prevent too many interations
        if(i>=imax):
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number =",i)
            
        #iterate
        i += 1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i, xi, h_step, b)
        print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = 0
            
    return x,y

In [25]:
a = 0.0
b = 2.0*np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform the integration
x,y = rk4_mv(dydx,a,b,y_0,tolerance)

i =   1	x = 0.00062832	h = 0.00062832	b=6.28318531
i =   2	x = 0.00188496	h = 0.00125664	b=6.28318531
i =   3	x = 0.00439823	h = 0.00251327	b=6.28318531
i =   4	x = 0.00942478	h = 0.00502655	b=6.28318531
i =   5	x = 0.01947787	h = 0.01005310	b=6.28318531
i =   6	x = 0.03958407	h = 0.02010619	b=6.28318531
i =   7	x = 0.07979645	h = 0.04021239	b=6.28318531
i =   8	x = 0.16022123	h = 0.08042477	b=6.28318531
i =   9	x = 0.32107077	h = 0.16084954	b=6.28318531
i =  10	x = 0.46816761	h = 0.14709684	b=6.28318531
i =  11	x = 0.60535502	h = 0.13718741	b=6.28318531
i =  12	x = 0.74522296	h = 0.13986794	b=6.28318531
i =  13	x = 0.88873209	h = 0.14350913	b=6.28318531
i =  14	x = 1.02700188	h = 0.13826979	b=6.28318531
i =  15	x = 1.16350434	h = 0.13650247	b=6.28318531
i =  16	x = 1.29828215	h = 0.13477781	b=6.28318531
i =  17	x = 1.43207856	h = 0.13379641	b=6.28318531
i =  18	x = 1.56536018	h = 0.13328163	b=6.28318531
i =  19	x = 1.69861470	h = 0.13325451	b=6.28318531
i =  20	x = 1.83231473	h = 0.13